In [1]:
!pip install category-encoders


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 2.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import requests as rq
import json as js
from time import sleep
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
import category_encoders as ce
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
warnings.filterwarnings('ignore')
import joblib

# API Call

In [4]:
class BestTimeToFertilize:
    __BASE_URL = "https://api.weatherbit.io/v2.0/forecast/daily?"
    __API_KEY = "051d187cb45f4da7835e3fcd695dc707"

    def __init__(self, city_name = 'Nagpur', state_name = 'Maharashtra', days = 7):
        self.city_name = '+'.join(city_name.lower().strip().split())
        self.state_name = '+'.join(state_name.lower().strip().split())
        self.country_name = 'IN'
        self.days = days
        self.response = None
        self.response_code = None
        self.weather_data = list()

    def api_caller(self):
        try:
            complete_url = "{0}city={1}&state={2}&country={3}&key={4}&days={5}".format(self.__BASE_URL, self.city_name, self.state_name, self.country_name, self.__API_KEY, self.days)
            print(complete_url)
            self.response = rq.get(complete_url)
            sleep(5)
            self.response_code = self.response.status_code
            return self.response_code
        except Exception as msg:
            print("api_caller():", msg)
            return -1

    def is_api_call_success(self):
        if self.response_code == 200:
            return True
        elif self.response_code == 204:
            print('Oops! It seems there was an issue with the API call. Please check your input and try again later.')
        return False

    def json_file_bulider(self):
        try:
            json_obj = self.response.json()
            with open('weather_data.json', 'w') as file:
                js.dump(json_obj, file, indent = 1, sort_keys = True)
            print("weather_data.json file built successfully")
        except Exception as msg:
            print("json_bulider():", msg)

    def best_time_fertilize(self):
        json_obj = self.response.json()

        print("City:", json_obj['city_name'], "\n")

        prolonged_precip = 0
        prolonged_prob = 0
        heavy_rain_2d = False
        heavy_rain_chance_2d = 0
        precip_2d = 0
        precip_chance_2d = 0

        for i in range(self.days):
            date = json_obj['data'][i]['datetime']
            temp = json_obj['data'][i]['temp']
            rh = json_obj['data'][i]['rh']
            precip = json_obj['data'][i]['precip']
            prob = json_obj['data'][i]['pop']
            w_code = json_obj['data'][i]['weather']['code']
            w_desc = json_obj['data'][i]['weather']['description']
            i_code = json_obj['data'][i]['weather']['icon']
            prolonged_precip += precip
            prolonged_prob += prob

            count_2d = 0
            if i < 2:
                precip_2d += precip
                precip_chance_2d += prob
                if w_code in [202, 233, 502, 521, 522]:
                    heavy_rain_2d = True
                    heavy_rain_chance_2d += prob
                    count_2d += 1
                    heavy_rain_chance_2d //= count_2d

            di = {
                  "Date": str(date),
                  "Temperature": str(temp),
                  "Relative Humidity": str(rh),
                  "Rainfall": str(precip),
                  "Probability of Precipitation": str(prob),
                  "Weather code": str(w_code),
                  "Weather Description": str(w_desc),
                  "Icon code": str(i_code)
                 }
            self.weather_data.append(di)

            print("Date:", date)
            print("Temperature:", temp)
            print("Relative Humidity:", rh)
            print("Rainfall:", precip)
            print("Probability of Precipitation:", prob)
            print("Weather code:", w_code, "->", w_desc)
            print()

        prolonged_prob //= self.days
        precip_chance_2d //= 2

        print("-"*80)
        print("Rainfall Forecast for the Next 2 Days (including today):", precip_2d)
        print("Probability of Rain for the Next 2 Days (including today):", precip_chance_2d)
        print()

        if heavy_rain_2d:
            print("*"*21, "Warning !!!", "*"*21)
            print("Heavy Rain Chances:", heavy_rain_chance_2d)
            print("Heavy Rainfall puts your fertilizer at risk.")
            print("*"*21, "Warning !!!", "*"*21)

        print("Prolonged Precipitation:", prolonged_precip)
        print("Prolonged Precipitation Probability:", prolonged_prob)
        print()

        if prolonged_precip > 12.7 and prolonged_prob >= 50:
            print("*"*21, "Warning !!!", "*"*21)
            print("Prolonged Rainfall of greater than 12.7 mm puts your fertilizer at risk.")
            print("*"*21, "Warning !!!", "*"*21)

In [5]:
if __name__ == '__main__':
    obj = BestTimeToFertilize('Mumbai', 'Maharashtra')
    print(obj.api_caller())
    if obj.is_api_call_success():
        obj.best_time_fertilize()
        obj.json_file_bulider()
#         print(obj.weather_data)

https://api.weatherbit.io/v2.0/forecast/daily?city=mumbai&state=maharashtra&country=IN&key=051d187cb45f4da7835e3fcd695dc707&days=7
200
City: Mumbai 

Date: 2023-10-19
Temperature: 31
Relative Humidity: 57
Rainfall: 0
Probability of Precipitation: 0
Weather code: 801 -> Few clouds

Date: 2023-10-20
Temperature: 30.1
Relative Humidity: 68
Rainfall: 0.125
Probability of Precipitation: 20
Weather code: 801 -> Few clouds

Date: 2023-10-21
Temperature: 30.2
Relative Humidity: 65
Rainfall: 0
Probability of Precipitation: 0
Weather code: 801 -> Few clouds

Date: 2023-10-22
Temperature: 29.9
Relative Humidity: 66
Rainfall: 0
Probability of Precipitation: 0
Weather code: 801 -> Few clouds

Date: 2023-10-23
Temperature: 29.6
Relative Humidity: 61
Rainfall: 0
Probability of Precipitation: 0
Weather code: 801 -> Few clouds

Date: 2023-10-24
Temperature: 29.3
Relative Humidity: 59
Rainfall: 0
Probability of Precipitation: 0
Weather code: 801 -> Few clouds

Date: 2023-10-25
Temperature: 29.1
Relative

# Data

In [12]:
# data = './app/data/Nutrient_recommendation.csv'
data = '/content/Nutrient_recommendation.csv'
df = pd.read_csv(data, header=None)
df.shape

(2201, 7)

In [13]:
df.head(10)

,0,1,2,3,4,5,6
0,Crop,Temperature,Humidity,Rainfall,Label_N,Label_P,Label_K
1,rice,20.87974371,82.00274423,202.9355362,90,42,43
2,rice,21.77046169,80.31964408,226.6555374,85,58,41
3,rice,23.00445915,82.3207629,263.9642476,60,55,44
4,rice,26.49109635,80.15836264,242.8640342,74,35,40
5,rice,20.13017482,81.60487287,262.7173405,78,42,42
6,rice,23.05804872,83.37011772,251.0549998,69,37,42
7,rice,22.70883798,82.63941394,271.3248604,69,55,38
8,rice,20.27774362,82.89408619,241.9741949,94,53,40
9,rice,24.51588066,83.5352163,230.4462359,89,54,38


In [14]:
df.columns = ['Crop', 'Temperature', 'Humidity', 'Rainfall', 'Label_N', 'Label_P', 'Label_K']
df.drop(df.index[:1], inplace=True)
df.head(10)

,Crop,Temperature,Humidity,Rainfall,Label_N,Label_P,Label_K
1,rice,20.87974371,82.00274423,202.9355362,90,42,43
2,rice,21.77046169,80.31964408,226.6555374,85,58,41
3,rice,23.00445915,82.3207629,263.9642476,60,55,44
4,rice,26.49109635,80.15836264,242.8640342,74,35,40
5,rice,20.13017482,81.60487287,262.7173405,78,42,42
6,rice,23.05804872,83.37011772,251.0549998,69,37,42
7,rice,22.70883798,82.63941394,271.3248604,69,55,38
8,rice,20.27774362,82.89408619,241.9741949,94,53,40
9,rice,24.51588066,83.5352163,230.4462359,89,54,38
10,rice,23.22397386,83.03322691,221.2091958,68,58,38


In [15]:
df[['Temperature', 'Humidity', 'Rainfall']] = df[['Temperature', 'Humidity', 'Rainfall']].astype(float)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 1 to 2200
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Crop         2200 non-null   object 
 1   Temperature  2200 non-null   float64
 2   Humidity     2200 non-null   float64
 3   Rainfall     2200 non-null   float64
 4   Label_N      2200 non-null   object 
 5   Label_P      2200 non-null   object 
 6   Label_K      2200 non-null   object 
dtypes: float64(3), object(4)
memory usage: 120.4+ KB


In [17]:
df.isnull().sum()

Crop           0
Temperature    0
Humidity       0
Rainfall       0
Label_N        0
Label_P        0
Label_K        0
dtype: int64

In [18]:
X = df.drop(['Label_N', 'Label_P', 'Label_K'], axis=1)

y1 = df['Label_N']
y2 = df['Label_P']
y3 = df['Label_K']

y = np.concatenate([[y1], [y2], [y3]], axis=0).T
y

array([['90', '42', '43'],
       ['85', '58', '41'],
       ['60', '55', '44'],
       ...,
       ['118', '33', '30'],
       ['117', '32', '34'],
       ['104', '18', '30']], dtype=object)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
X_train.shape, X_test.shape

((1760, 4), (440, 4))

In [20]:
y_train.shape, y_test.shape

((1760, 3), (440, 3))

In [21]:
y1_train = y_train[:, 0]
y2_train = y_train[:, 1]
y3_train = y_train[:, 2]

y1_test = y_test[:, 0]
y2_test = y_test[:, 1]
y3_test = y_test[:, 2]

In [22]:
X_train.head(10)

,Crop,Temperature,Humidity,Rainfall
595,mothbeans,28.698413,61.147544,65.675918
1080,banana,28.227767,80.643038,98.004030
1782,papaya,29.607188,93.156428,62.687105
338,kidneybeans,20.530527,18.092240,120.450929
1893,coconut,26.916419,99.846716,225.632366
1340,watermelon,25.504055,81.402974,48.478333
1474,muskmelon,28.585465,91.892168,26.883726
1232,grapes,16.762017,82.003356,73.287128
96,rice,22.683191,83.463583,194.265172
31,rice,21.446540,84.943760,272.201720


In [23]:
mapping = dict()
# with open("./app/data/mapped_crops.csv", "w") as f:
with open("/content/mapped_crops.csv", "w") as f:
    f.write("Crops,Key\n")
    for i, crop in enumerate(np.unique(df[['Crop']]), 1):
        mapping[crop] =  i
        f.write("%s,%d\n" % (crop, i))
    mapping['NA'] = np.nan
    f.write("NA,nan")

print(mapping)

{'apple': 1, 'banana': 2, 'blackgram': 3, 'chickpea': 4, 'coconut': 5, 'coffee': 6, 'cotton': 7, 'grapes': 8, 'jute': 9, 'kidneybeans': 10, 'lentil': 11, 'maize': 12, 'mango': 13, 'mothbeans': 14, 'mungbean': 15, 'muskmelon': 16, 'orange': 17, 'papaya': 18, 'pigeonpeas': 19, 'pomegranate': 20, 'rice': 21, 'watermelon': 22, 'NA': nan}


In [24]:
ordinal_cols_mapping = [{"col": "Crop", "mapping": mapping}, ]
encoder = ce.OrdinalEncoder(cols = 'Crop', mapping = ordinal_cols_mapping, return_df = True)

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)
X_train.head()

,Crop,Temperature,Humidity,Rainfall
595,14,28.698413,61.147544,65.675918
1080,2,28.227767,80.643038,98.004030
1782,18,29.607188,93.156428,62.687105
338,10,20.530527,18.092240,120.450929
1893,5,26.916419,99.846716,225.632366


# Training

### Random Forest

In [25]:
from sklearn.ensemble import  RandomForestRegressor as rfr
# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 4, 6]
}

regressor = rfr(random_state=42)

grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y1_train)

best_params_rf = grid_search.best_params_
best_regressor_rf = grid_search.best_estimator_

# print("Best Regressor : ",best_regressor_rf)
# print("Best Parameters : ",best_params_rf)

In [26]:
print(" Using Random Forest Regressor \n")
# For Nitrogen

rf1 = rfr(**best_params_rf)
rf1.fit(X_train, y1_train)
y1_pred = rf1.predict(X_test)

print(" For Nitrogen ")

print('Mean Absolute Error:', metrics.mean_absolute_error(y1_test, y1_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y1_test, y1_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)), '.3f'))
print("RMSE: ", rmse)
print()

# For Phosphorus

rf2 = rfr(**best_params_rf)
rf2.fit(X_train, y2_train)
y2_pred = rf2.predict(X_test)

print(" For Phosphorus ")

print('Mean Absolute Error:', metrics.mean_absolute_error(y2_test, y2_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y2_test, y2_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)), '.3f'))
print("RMSE: ", rmse)
print()

# For Potassium

rf3 = rfr(**best_params_rf)
rf3.fit(X_train, y3_train)
y3_pred = rf3.predict(X_test)

print(" For Potassium ")

print('Mean Absolute Error:', metrics.mean_absolute_error(y3_test, y3_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y3_test, y3_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)), '.3f'))
print("RMSE: ", rmse)
print()

 Using Random Forest Regressor 

 For Nitrogen 
Mean Absolute Error: 11.030359598792717
Mean Squared Error: 164.91279442405607
Root Mean Squared Error: 12.841837657596209
RMSE:  12.842

 For Phosphorus 
Mean Absolute Error: 7.309249633590479
Mean Squared Error: 78.02819950553123
Root Mean Squared Error: 8.833357204683349
RMSE:  8.833

 For Potassium 
Mean Absolute Error: 2.8647698491534905
Mean Squared Error: 11.837379176880967
Root Mean Squared Error: 3.4405492551162475
RMSE:  3.441



### Gradient Boosting

In [48]:
from sklearn.ensemble import GradientBoostingRegressor as xgboost
from xgboost import XGBRegressor

# Define the hyperparameter grid for GridSearch
param_grid = {
    'n_estimators': [50, 75, 100, 125],
    'max_depth': [5, 10, 15, 20],
}

gbr = XGBRegressor(random_state=42)

grid_search = GridSearchCV(estimator=gbr, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y1_train)

best_params_gb = grid_search.best_params_
best_regressor_gb = grid_search.best_estimator_

# print("Best Regressor : ", best_regressor_gb)
# print("Best Parameter : ", best_params_gb)


In [49]:
print(" Using Gradient Boosting Regressor \n")
# For Nitrogen

gb1 = XGBRegressor(**best_params_gb)
gb1.fit(X_train, y1_train)
y1_pred = gb1.predict(X_test)

print(" For Nitrogen ")

print('Mean Absolute Error:', metrics.mean_absolute_error(y1_test, y1_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y1_test, y1_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)), '.3f'))
print("RMSE : ", rmse)
print()

# For Phosphorus

gb2 = XGBRegressor(**best_params_gb)
gb2.fit(X_train, y2_train)
y2_pred = gb2.predict(X_test)

print(" For Phosphorus ")

print('Mean Absolute Error:', metrics.mean_absolute_error(y2_test, y2_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y2_test, y2_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)), '.3f'))
print("RMSE: ", rmse)
print()

# For Potassium

gb3 = XGBRegressor(**best_params_gb)
gb3.fit(X_train, y2_train)
y2_pred = gb3.predict(X_test)

print(" For Potassium")

print('Mean Absolute Error:', metrics.mean_absolute_error(y3_test, y3_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y3_test, y3_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)), '.3f'))
print("RMSE: ", rmse)
print()

 Using Gradient Boosting Regressor 

 For Nitrogen 
Mean Absolute Error: 10.962219138578934
Mean Squared Error: 170.9457447826295
Root Mean Squared Error: 13.074622165960648
RMSE :  13.075

 For Phosphorus 
Mean Absolute Error: 7.397301879796115
Mean Squared Error: 77.07922933236082
Root Mean Squared Error: 8.779477736879388
RMSE:  8.779

 For Potassium
Mean Absolute Error: 2.8647698491534905
Mean Squared Error: 11.837379176880967
Root Mean Squared Error: 3.4405492551162475
RMSE:  3.441



### Light GBM

In [27]:
from lightgbm import LGBMRegressor as lgbm

param_grid = {
    'n_estimators': [50, 75, 100, 125],
    'max_depth': [5, 10, 15, 20],
    'min_child_samples': [2, 4, 6, 8],
    'min_child_weight': [1e-4, 1e-3, 1e-2, 1e-1],
    'num_leaves' : [31]
}

regressor = lgbm(random_state=42)

grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y1_train)

best_params_lgb = grid_search.best_params_
best_regressor_lgb = grid_search.best_estimator_

# print("Best Parameters : ",best_regressor_lgb)
# print("Best Parameters : ", best_params_lgb)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000128 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 788
[LightGBM] [Info] Number of data points in the train set: 1760, number of used features: 4
[LightGBM] [Info] Start training from score 50.463636
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

In [28]:
print(" Using Light GBM Regressor \n")
# For Nitrogen

lgb1 = lgbm(**best_params_lgb)
lgb1.fit(X_train, y1_train)
y1_pred = lgb1.predict(X_test)

print(" For Nitrogen ")

print('Mean Absolute Error:', metrics.mean_absolute_error(y1_test, y1_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y1_test, y1_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)), '.3f'))
print("RMSE : ", rmse)
print()

# For Phosphorus

lgb2 = lgbm(**best_params_lgb)
lgb2.fit(X_train, y2_train)
y2_pred = lgb2.predict(X_test)

print(" For Phosphorus ")

print('Mean Absolute Error:', metrics.mean_absolute_error(y2_test, y2_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y2_test, y2_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)), '.3f'))
print("RMSE: ", rmse)
print()

# For Potassium

lgb3 = lgbm(**best_params_lgb)
lgb3.fit(X_train, y2_train)
y2_pred = lgb3.predict(X_test)

print(" For Potassium")

print('Mean Absolute Error:', metrics.mean_absolute_error(y3_test, y3_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y3_test, y3_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)), '.3f'))
print("RMSE: ", rmse)
print()

 Using Light GBM Regressor 

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000061 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 788
[LightGBM] [Info] Number of data points in the train set: 1760, number of used features: 4
[LightGBM] [Info] Start training from score 50.463636
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

Robust Regression (RANSAC)


In [43]:
from sklearn.linear_model import LinearRegression, RANSACRegressor

# Define the hyperparameter grid for GridSearch
param_grid = {
    'min_samples' : [40, 50, 60, 70, 80],
    'max_trials' : [50, 75, 100, 125, 150],
    'residual_threshold' : [5, 10, 15, 20, 25]
}

rrr = RANSACRegressor(random_state=42,
                      base_estimator=LinearRegression())

grid_search = GridSearchCV(estimator=rrr, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y1_train)

best_params_rrr = grid_search.best_params_
best_regressor_rrr = grid_search.best_estimator_

# print("Best Regressor : ", best_regressor_rrr)
# print("Best parameters : ", best_params_rrr)

In [44]:
print(" Using Robust Regressor \n")
# For Nitrogen

rr1 = RANSACRegressor(**best_params_rrr)
rr1.fit(X_train, y1_train)
y1_pred = rr1.predict(X_test)

print(" For Nitrogen ")

print('Mean Absolute Error:', metrics.mean_absolute_error(y1_test, y1_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y1_test, y1_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)), '.3f'))
print("RMSE : ", rmse)
print()

# For Phosphorus

rr2 = RANSACRegressor(**best_params_rrr)
rr2.fit(X_train, y2_train)
y2_pred = rr2.predict(X_test)

print(" For Phosphorus ")

print('Mean Absolute Error:', metrics.mean_absolute_error(y2_test, y2_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y2_test, y2_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)), '.3f'))
print("RMSE: ", rmse)
print()

# For Potassium

rr3 = RANSACRegressor(**best_params_rrr)
rr3.fit(X_train, y2_train)
y2_pred = rr3.predict(X_test)

print(" For Potassium")

print('Mean Absolute Error:', metrics.mean_absolute_error(y3_test, y3_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y3_test, y3_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)), '.3f'))
print("RMSE: ", rmse)
print()

 Using Robust Regressor 

 For Nitrogen 
Mean Absolute Error: 31.542620882403757
Mean Squared Error: 1488.921436826634
Root Mean Squared Error: 38.58654476403185
RMSE :  38.587

 For Phosphorus 
Mean Absolute Error: 22.777668687526276
Mean Squared Error: 922.6586494752579
Root Mean Squared Error: 30.375296697732153
RMSE:  30.375

 For Potassium
Mean Absolute Error: 2.8647698491534905
Mean Squared Error: 11.837379176880967
Root Mean Squared Error: 3.4405492551162475
RMSE:  3.441



### ElasticNet

In [45]:
from sklearn.linear_model import ElasticNet

param_grid = {
    'alpha': [0.1, 1.0, 10.0, 100.0],
    'l1_ratio': [0.1, 0.35, 0.65, 0.95],
}

regressor = ElasticNet(random_state=42)

grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_params_en = grid_search.best_params_
best_regressor_en = grid_search.best_estimator_

# print("Best Regressor : ", best_regressor_en)
# print("Best Parameters : ",best_params_en)

In [46]:
print(" Using Elastic Net \n")
# For Nitrogen

en1 = ElasticNet(**best_params_en)
en1.fit(X_train, y1_train)
y1_pred = en1.predict(X_test)

print(" For Nitrogen ")

print('Mean Absolute Error:', metrics.mean_absolute_error(y1_test, y1_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y1_test, y1_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)), '.3f'))
print("RMSE : ", rmse)
print()

# For Phosphorus

en2 = ElasticNet(**best_params_en)
en2.fit(X_train, y2_train)
y2_pred = en2.predict(X_test)

print(" For Phosphorus ")

print('Mean Absolute Error:', metrics.mean_absolute_error(y2_test, y2_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y2_test, y2_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)), '.3f'))
print("RMSE: ", rmse)
print()

# For Potassium

en3 = ElasticNet(**best_params_en)
en3.fit(X_train, y2_train)
y2_pred = en3.predict(X_test)

print(" For Potassium")

print('Mean Absolute Error:', metrics.mean_absolute_error(y3_test, y3_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y3_test, y3_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y3_test, y3_pred)), '.3f'))
print("RMSE: ", rmse)
print()

 Using Elastic Net 

 For Nitrogen 
Mean Absolute Error: 31.807294104287006
Mean Squared Error: 1338.3409657399445
Root Mean Squared Error: 36.583342736004106
RMSE :  36.583

 For Phosphorus 
Mean Absolute Error: 22.662922836608644
Mean Squared Error: 889.7132877339687
Root Mean Squared Error: 29.828062084788023
RMSE:  29.828

 For Potassium
Mean Absolute Error: 2.8647698491534905
Mean Squared Error: 11.837379176880967
Root Mean Squared Error: 3.4405492551162475
RMSE:  3.441



###Average Ensemble Models


In [50]:
model_1 = RANSACRegressor()
model_2 = XGBRegressor()
model_3 = rfr()
model_4 = lgbm()
model_5 = ElasticNet()

# training all the model on the training dataset
model_1.fit(X_train, y_train)
model_2.fit(X_train, y_train)
model_3.fit(X_train, y_train)
model_4.fit(X_train, y_train)
model_5.fit(X_train, y_train)

# predicting the output on the validation dataset
pred_1 = model_1.predict(X_test)
pred_2 = model_2.predict(X_test)
pred_3 = model_3.predict(X_test)
pred_4 = model_4.predict(X_test)
pred_5 = model_5.predict(X_test)

# final prediction after averaging on the prediction of all 3 models
pred_final = (pred_1 + pred_2 + pred_3 + pred_4 + pred_5)/3.0


TypeError: ignored

In [51]:
print(" Using Elastic Net \n")
# For Nitrogen

en1 = ElasticNet(**best_params_en)
en1.fit(X_train, y1_train)
y1_pred = en1.predict(X_test)

print(" For Nitrogen ")

print('Mean Absolute Error:', metrics.mean_absolute_error(y1_test, y1_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y1_test, y1_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)), '.3f'))
print("RMSE : ", rmse)
print()

 Using Elastic Net 

 For Nitrogen 
Mean Absolute Error: 31.807294104287006
Mean Squared Error: 1338.3409657399445
Root Mean Squared Error: 36.583342736004106
RMSE :  36.583



### Best Model

In [29]:
!pip install hickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 3.4 MB/s eta 0:00:00


In [42]:
import pickle as pkl
# save the model to disk
joblib.dump(rf1, '/content/model/rf_N-v1.pkl')
joblib.dump(rf2, '/content/model/lgbm_P-v1.pkl')
joblib.dump(rf3, '/content/model/rf_K-v1.pkl')

['/content/model/rf_K-v1.pkl']